In [0]:
# NACHISS - CLEAN 

---------

# Enhanced BiGram model

----------

### Objective is correctness and clarity of concepts and not efficiency

#### Author : Anuj



------
------

IMPROVEMENTS:     
1) Better Tokenization      

LEFTOVERS:   

1) ~~Tensor board graphs for loss~~  

2) Tensor board for network vis      
   (a)   ~~Draw the Plot~~   
   (b)   Interpret and Map the network to Code     

3) Is embedding good or not ?    

4) Embedding visualization ?   
    (a) ~~Plot the vector in an interactive graph~~     
    (b) ~~Plot vectors in Tensorboard~~    
    (c) ~~After every k epochs, take some random words and show their nearest neighbours~~     
    (d) Show vectors being pulled appart in different direction    
    (e) Analogy tasks     
    (f) validation set loss   

5) model perplexity     

6) Input vectors and output vectors     

OPTIMIZATIONS     

1) Better Optimizer     
2) negative sampling    
3) manage learning rate     
4) Param initializations      
     

------
------


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [0]:
import sys
import tensorflow as tf
import numpy as np
from collections import Counter
import random

In [0]:
sys.path.append("/content/drive/NLP_bootcamp/Data/")


In [5]:
pwd

'/content'

# Load the data file 

In [0]:
path = "/content/drive/NLP_bootcamp/Data/harry_potter_3.txt"
text = open(path).read()

raw_data = text.replace('\r\n', ' ')



In [0]:
tokens = raw_data.split()


## frequency counts

In [0]:
# count the frequency of tokens, pick top 100
counts = Counter(tokens)
top_100 = counts.most_common(100)

In [9]:
top_100

[('the', 4578),
 ('to', 2510),
 ('and', 2392),
 ('of', 1990),
 ('a', 1965),
 ('was', 1590),
 ('his', 1401),
 ('said', 1263),
 ('he', 1257),
 ('Harry', 1256),
 ('in', 1112),
 ('had', 865),
 ('at', 761),
 ('you', 717),
 ('--', 710),
 ('I', 706),
 ('it', 684),
 ('that', 667),
 ('on', 621),
 ('with', 620),
 ('as', 590),
 ('for', 487),
 ('He', 464),
 ('out', 427),
 ('Hermione', 402),
 ('Ron', 396),
 ('were', 385),
 ('they', 384),
 ('be', 376),
 ('him', 372),
 ('up', 356),
 ('from', 356),
 ('into', 353),
 ('Professor', 342),
 ('have', 331),
 ('but', 322),
 ('her', 321),
 ('--"', 319),
 ('The', 310),
 ('all', 285),
 ('back', 282),
 ('been', 264),
 ('very', 252),
 ('Harry,', 246),
 ('what', 245),
 ('"I', 241),
 ('who', 240),
 ('their', 237),
 ('looked', 234),
 ('them', 231),
 ('this', 222),
 ('down', 219),
 ('one', 214),
 ('like', 214),
 ('about', 214),
 ('she', 211),
 ('Lupin', 210),
 ('could', 209),
 ('got', 205),
 ('not', 201),
 ('an', 199),
 ('is', 199),
 ('over', 194),
 ('around', 190),
 

## Build token to Id mapping

In [0]:
word_to_id = {}

for token in set(tokens):
    word_to_id[token] = len(word_to_id)


In [0]:
id_to_word = dict(zip(word_to_id.values(), word_to_id.keys()))


# Convert Data to IDs

In [0]:
data = []

data = [word_to_id[tok] for tok in tokens]

In [13]:
print (tokens[:20])

for id in data[:20]:
    print (id_to_word[id]) 

['Harry', 'Potter', 'and', 'the', 'Prisoner', 'of', 'Azkaban', 'by', 'J.K.', 'Rowling', 'CHAPTER', 'ONE', 'OWL', 'POST', 'Harry', 'Potter', 'was', 'a', 'highly', 'unusual']
Harry
Potter
and
the
Prisoner
of
Azkaban
by
J.K.
Rowling
CHAPTER
ONE
OWL
POST
Harry
Potter
was
a
highly
unusual


## Build validation set - randomly choose a batch from these 100

In [0]:
top_100_ids = []

for key, count in top_100:
    top_100_ids.append(word_to_id[key])
    
    
#print top_100_ids

In [0]:
validation_size = 32
validation_set = random.sample(top_100_ids, validation_size)

In [0]:
#top_100_ids

## Now lets slice the data into training set

In [0]:
class BatchData():
    
    def __init__(self, batch_size, list_of_token_ids):
        self.batch_size_ = batch_size
        self.data_ = list_of_token_ids
        self.data_len_ = len(self.data_)
        self.global_pointer = 0
        
    def getBatch(self):
         
        if self.global_pointer + self.batch_size_ > self.data_len_:
            print ("Abort")
            
        X = np.zeros(shape=(self.batch_size_,), dtype=np.int32)
        Y = np.zeros(shape=(self.batch_size_,), dtype=np.int32)
        
        for i in range(self.batch_size_-1):
            
            X[i] = self.data_[self.global_pointer]
            Y[i] = self.data_[self.global_pointer+1]
            
            self.global_pointer = (self.global_pointer + 1)%self.data_len_
            
            
        return X, Y
    
    def reset(self):
        
        self.global_pointer = 0
    

In [0]:
#batch = BatchData(batch_size=32, list_of_token_ids=data)

In [0]:
#X, Y = batch.getBatch()

In [0]:
#print (X)
#print (Y)

In [0]:
#data[:64]

# Network

In [0]:
batch_size=32
num_batches = len(data)/batch_size

embedding_dims = 128
vocab_size = len(set(tokens))

In [0]:
X = tf.placeholder(shape=(batch_size,), dtype=tf.int32, name='X_var')
Y = tf.placeholder(shape=(batch_size,), dtype=tf.int32, name='Y_var')
valid_X = tf.Variable(validation_set, dtype=tf.int32, name='X_valid')

In [0]:
y_oh = tf.one_hot(indices=X, depth=vocab_size, name='Converting_Y_to_Y_oh')

In [25]:
print (X.get_shape())
print (Y.get_shape())
#print (y_oh.get_shape())

(32,)
(32,)


In [0]:
embedding_layer_1 = tf.Variable(tf.truncated_normal(shape=(vocab_size, embedding_dims),mean=0.0, stddev=1.0, dtype=tf.float32), name="Embeddings_Matrix")    
embeded = tf.nn.embedding_lookup(embedding_layer_1, ids=X, name="Embedding_LookUp")

In [27]:
embeded.get_shape()

TensorShape([Dimension(32), Dimension(128)])

In [0]:
#softmax weights, bias
W = tf.Variable(tf.truncated_normal(shape=(embedding_dims, vocab_size),mean=0.0, stddev=1.0, dtype=tf.float32), name="Softmax_Weights_Matrix")
b = tf.Variable(tf.zeros(shape=(vocab_size,)), name="Softmax_Bias_Vector")


In [29]:
logits = tf.add(tf.matmul(embeded, W, name="WX"), b, name="WX_plus_b")

#logits = tf.add(tf.matmul(embed, softmax_weights, name="WX"), softmax_bias, name="WX_plus_b")

loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_oh, name="Compute_Loss")
#mean_loss = tf.reduce_mean(loss)
mean_loss = tf.reduce_mean(loss, name="Compute_mean_loss")

tf.summary.scalar("mean_loss", mean_loss)

<tf.Tensor 'mean_loss:0' shape=() dtype=string>

In [30]:
print (logits.get_shape())
print (y_oh.get_shape())

(32, 15519)
(32, 15519)


In [0]:
optimizer = tf.train.GradientDescentOptimizer(0.5, name="Optimizer").minimize(mean_loss)

In [0]:
summary_op = tf.summary.merge_all()

In [0]:
#compute L2 norm for cosine similarity
norm = tf.sqrt(tf.reduce_sum(tf.square(embedding_layer_1), axis=1, keepdims=True))
normalised_embeddings = embedding_layer_1 / norm

# get validation set embeddings
validation_data_embeddings = tf.nn.embedding_lookup(normalised_embeddings, ids=valid_X, name="validation_embeddings_lookup")

# similarity score of validation embeddings w.r.t normalised= dot product between validation_data_embeddings and mornalised embeddings
similarity = tf.matmul(validation_data_embeddings, tf.transpose(normalised_embeddings))  # C.A = C x transpose(A)

# Training

In [34]:
num_of_epochs = 5
LOG_DIR = './bigram_chk_pts'

# A SIMPLE saver() to save the model
saver = tf.train.Saver()

batch = BatchData(batch_size=32, list_of_token_ids=data)

with tf.Session() as sess:
    
    # writer to write graph to tensorboard
    writer = tf.summary.FileWriter(LOG_DIR, sess.graph)

    tf.global_variables_initializer().run()
    print ("initialised")

    for epoch_id in range(num_of_epochs):

        av_batch_loss = 0

        for batch_id in range(int(num_batches)):

            X_, Y_ = batch.getBatch()

            feed_dict = {}
            feed_dict[X] = X_
            feed_dict[Y] = Y_

            batch_loss, _, summary = sess.run([mean_loss, optimizer,summary_op], feed_dict=feed_dict)
            
            #writer.add_summary(batch_loss, epoch) 
            writer.add_summary(summary, global_step=epoch_id)

            av_batch_loss += batch_loss
            
            if batch_id % 10000 == 0:
                print ("For epoch = %d, batch id = %d, batch loss = %f" %(epoch_id, batch_id, batch_loss))
                
                #print validation data
                sim = similarity.eval() # compute similarity
                
                #iterate over each validation example
                for i in range(validation_size):
                    word = id_to_word[validation_set[i]]
                    top_k = 8
                    # sort indexes and pick top k. we take 1:top_k+1 since 0th top pick will the same word itself
                    nearest = (sim[i,:].argsort()[1:top_k+1])
                    log = '\t Nearest to %s : ' %word
                    for k in range(top_k):
                        nearest_word = id_to_word[nearest[k]]
                        log = '%s %s,' %(log, nearest_word)
                    print (log)
                        
                    
                    

        print ("\nFor epoch = %d, Av loss = %f\n\n" %(epoch_id, av_batch_loss/num_batches))
        
        batch.reset()
        
    save_path = saver.save(sess, LOG_DIR)
    print("Model saved in file: %s" % save_path)
        
        
        
        

initialised
For epoch = 0, batch id = 0, batch loss = 33.827858
	 Nearest to Snape :  "Oh, trousers, rope,, orders, quiet, Broken, wasn't., Owl,
	 Nearest to do :  accusing, master,, furniture,, magnified, forgetful, sighted, crossed, LILY,
	 Nearest to they :  pleasant, gang., lightning, Dangerous., freedom., Mum, prone, lonely.,
	 Nearest to him. :  Arthur, danger., subjects,, Hogsmeade.", Hog, inch,, school, about!",
	 Nearest to to :  tugging, "Exactly!, great!", *ignored, "Fudge'll, amusement, "Haven't, Hermlone,,
	 Nearest to But :  Lavender?", upstairs., called., fortyseven, disgruntled, squinting, dog...., sleep....,
	 Nearest to --" :  halt,, charge., Firebolt...", fourteen, "'Spect, mentally, broke., cabin,,
	 Nearest to who :  "Dear,, bars...., Law, night.", ride., cleaned, "Laughed,", jolt.,
	 Nearest to Harry :  "Easily, truth, disappointed, unicorn,, waited, "Firstly,", "THIRTY-ZERO!, will.",
	 Nearest to looked :  slept., corner, mercy...., sarcasm, Derrick., lead, Wizar

# Plot the Embeddings 

## Tensorboard way

In [35]:
#create metadata file

#reference: https://www.tensorflow.org/get_started/embedding_viz#metadata
LOG_DIR = './bigram_chk_pts'

file_name = LOG_DIR + '/metadata.tsv'
fp = open(file_name, 'w')

for i, v in enumerate(id_to_word):
    print (id_to_word[i])
    
    string = str(i) + '_' + id_to_word[i] + '\n'
    fp.write(string)
    print (string)
    
    #if i > 5:
    #    break
        
fp.close()

amazed
0_amazed

Granger,"
1_Granger,"

broodingly
2_broodingly

commands
3_commands

timidly.
4_timidly.

Reluctant
5_Reluctant

propelled
6_propelled

Shack,
7_Shack,

That's
8_That's

"This,
9_"This,

much."
10_much."

eating
11_eating

wilder;
12_wilder;

second,
13_second,

Blotts,
14_Blotts,

walls,
15_walls,

Belch
16_Belch

explore
17_explore

arm's
18_arm's

"Impervius!"
19_"Impervius!"

ages!"
20_ages!"

coward,
21_coward,

grassy
22_grassy

Marauder's
23_Marauder's

member.
24_member.

distract
25_distract

laugh,
26_laugh,

we?"
27_we?"

smashed
28_smashed

shout
29_shout

record,"
30_record,"

double
31_double

prodding
32_prodding

nightshirt
33_nightshirt

daylight
34_daylight

fairies.
35_fairies.

rounding
36_rounding

brow
37_brow

"Special
38_"Special

procession.
39_procession.

POTTER
40_POTTER

cars
41_cars

grip..."
42_grip..."

Minister,
43_Minister,

scratched
44_scratched

saying?"
45_saying?"

"Macnair,
46_"Macnair,

hurry,"
47_hurry,"

kiss,
48_kiss,

dislik

In [0]:
from tensorflow.contrib.tensorboard.plugins import projector
import os

summary_writer = tf.summary.FileWriter(LOG_DIR)

#get config params
config = projector.ProjectorConfig()

# You can add multiple embeddings. Here we add only one.
embedding = config.embeddings.add()
embedding.tensor_name = embedding_layer_1.name  # this is where you connect your embedding tensor

# Link this tensor to its metadata file (e.g. labels).
embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')

# The next line writes a projector_config.pbtxt in the LOG_DIR. TensorBoard will
# read this file during startup.
projector.visualize_embeddings(summary_writer, config)

# run your tensorboard
# tensorboard --logdir="./bigram_chk_pts" --port 6006